In [1]:
import requests
import json

key = 'enJH8FUX2z5Ar7NSCJvYI8pAIuDW0XDV9nbSkEMj'
url = 'https://api.meteo.cat/xema/v1' + '/estacions/X8/variables/mesurades/metadades'

headers = {'Accept': 'application/json', 'X-API-KEY': key}

r = requests.get(url, headers=headers)

r.json()

[{'codi': 1,
  'nom': 'Pressió atmosfèrica màxima',
  'unitat': 'hPa',
  'acronim': 'Px',
  'tipus': 'DAT',
  'decimals': 1,
  'estats': [{'codi': 2, 'dataInici': '2008-04-17T21:00Z', 'dataFi': None}],
  'basesTemporals': [{'codi': 'SH',
    'dataInici': '2008-04-17T21:00Z',
    'dataFi': None}]},
 {'codi': 2,
  'nom': 'Pressió atmosfèrica mínima',
  'unitat': 'hPa',
  'acronim': 'Pn',
  'tipus': 'DAT',
  'decimals': 1,
  'estats': [{'codi': 2, 'dataInici': '2008-04-17T13:00Z', 'dataFi': None}],
  'basesTemporals': [{'codi': 'SH',
    'dataInici': '2008-04-17T13:00Z',
    'dataFi': None}]},
 {'codi': 3,
  'nom': 'Humitat relativa màxima',
  'unitat': '%',
  'acronim': 'HRx',
  'tipus': 'DAT',
  'decimals': 0,
  'estats': [{'codi': 2, 'dataInici': '2009-05-19T14:30Z', 'dataFi': None}],
  'basesTemporals': [{'codi': 'SH',
    'dataInici': '2009-05-19T14:30Z',
    'dataFi': None}]},
 {'codi': 20,
  'nom': 'Velocitat del vent a 10 m (vec.)',
  'unitat': 'm/s',
  'acronim': 'VV10vec',
  'ti

In [2]:
key = 'enJH8FUX2z5Ar7NSCJvYI8pAIuDW0XDV9nbSkEMj'
url = 'https://api.meteo.cat/xema/v1' + '/estacions/mesurades/X8/2020/03/02'

headers = {'Accept': 'application/json', 'X-API-KEY': key}

r = requests.get(url, headers=headers)

data = r.json()[0]

variables = [data['variables'][i]['lectures'] for i in range(len(data['variables'])) if data['variables'][i]['codi'] == 35][0]

date_rain = [{'data':d['data'], 'valor':d['valor']} for d in variables ]

date_rain

[{'data': '2020-03-02T00:00Z', 'valor': 0},
 {'data': '2020-03-02T00:30Z', 'valor': 0},
 {'data': '2020-03-02T01:00Z', 'valor': 0},
 {'data': '2020-03-02T01:30Z', 'valor': 0},
 {'data': '2020-03-02T02:00Z', 'valor': 0},
 {'data': '2020-03-02T02:30Z', 'valor': 0.2},
 {'data': '2020-03-02T03:00Z', 'valor': 0.2},
 {'data': '2020-03-02T03:30Z', 'valor': 0},
 {'data': '2020-03-02T04:00Z', 'valor': 0},
 {'data': '2020-03-02T04:30Z', 'valor': 0},
 {'data': '2020-03-02T05:00Z', 'valor': 0},
 {'data': '2020-03-02T05:30Z', 'valor': 0},
 {'data': '2020-03-02T06:00Z', 'valor': 0},
 {'data': '2020-03-02T06:30Z', 'valor': 0},
 {'data': '2020-03-02T07:00Z', 'valor': 0},
 {'data': '2020-03-02T07:30Z', 'valor': 0.3},
 {'data': '2020-03-02T08:00Z', 'valor': 1},
 {'data': '2020-03-02T08:30Z', 'valor': 0},
 {'data': '2020-03-02T09:00Z', 'valor': 0},
 {'data': '2020-03-02T09:30Z', 'valor': 0},
 {'data': '2020-03-02T10:00Z', 'valor': 0},
 {'data': '2020-03-02T10:30Z', 'valor': 0},
 {'data': '2020-03-02T11:0

In [21]:
import datetime as dt 


dt.datetime.strptime(date_rain[5]['data'], '%Y-%m-%dT%H:%MZ').hour

2

In [3]:
import pandas as pd

gener = pd.read_csv('2021_01_Gener_BicingNou_INFORMACIO.csv')

gener.head()

,station_id,name,physical_configuration,lat,lon,altitude,address,post_code,capacity,nearby_distance,cross_street,last_updated,ttl
0,1,"GRAN VIA CORTS CATALANES, 760",ELECTRICBIKESTATION,41.397978,2.180107,16,"GRAN VIA CORTS CATALANES, 760",8013,46,1000,NaN,1609455629,25
1,2,"C/ ROGER DE FLOR, 126",ELECTRICBIKESTATION,41.395488,2.177198,17,"C/ ROGER DE FLOR, 126",8013,27,1000,NaN,1609455629,25
2,3,"C/ NÀPOLS, 82",ELECTRICBIKESTATION,41.394156,2.181331,11,"C/ NÀPOLS, 82",8013,27,1000,NaN,1609455629,25
3,4,"C/ RIBES, 13",ELECTRICBIKESTATION,41.393317,2.181248,8,"C/ RIBES, 13",8013,21,1000,NaN,1609455629,25
4,5,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",ELECTRICBIKESTATION,41.391103,2.180176,7,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",8018,39,1000,NaN,1609455629,25


In [4]:
locations = gener[['name', 'lat', 'lon']].groupby('name').mean().reset_index()

locations.loc[1]['lat']

locations.shape

(506, 3)

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import folium

from folium.plugins import HeatMap

map = folium.Map(location=[41.3870154, 2.1700471], zoom_start=13)
icon = folium.Icon(icon_size = (10,10))

for i in range(locations.shape[0]):
    folium.Marker([locations.loc[i]['lat'], locations.loc[i]['lon']], popup=locations.loc[i]['name'], icon = folium.Icon(color = 'red', icon = 'info_sign', icon_size=(15, 15))).add_to(map)

map


